## Imports


In [ ]:
import os
import sys
import pandas as pd
import altair as alt


In [ ]:
### package versions
packList=["pandas","altair","numpy","itkdb"]
for p in packList:
    if p in sys.modules.keys():
        mod=sys.modules.get(p)
        print(mod.__name__+" : "+mod.__version__)

In [ ]:
myList

In [ ]:
import pip._internal.operations.freeze
_ = pip._internal.operations.freeze.get_installed_distributions()
print(sorted(["%s==%s" % (i.key, i.version) for i in _])[:10])

## Read in Data
read raw format
 - ../exampleData.txt

In [ ]:
### list files in above directory
cwd = os.getcwd()
dataDir=cwd+"/../"
os.listdir(dataDir)

## Format raw data
Using pandas

## Visualise formatted data
tables and plots using pandas and altair